In [1]:
# Import packages
import pandas as pd
import re

# Set  number format
# np.set_printoptions(precision=6, suppress=True)
pd.options.display.float_format = "{:,.2f}".format


#### INDUSTRY x INDUSTRY Table

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
# ---------------------------------------------------------
dir_path = "../data/exiobase/_ixi/IOT_2019_ixi/"


# ---------------------------------------------------------
# IMPORT MATRICES
# ---------------------------------------------------------
# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL
# ---------------------------------------------------------
# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()

In [3]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]

Index(['Manure treatment (biogas), storage and land application',
       'Extraction of natural gas and services related to natural gas extraction, excluding surveying',
       'Extraction, liquefaction, and regasification of other petroleum and gaseous materials',
       'Production of electricity by gas',
       'Manufacture of gas; distribution of gaseous fuels through mains',
       'Biogasification of food waste, incl. land application',
       'Biogasification of paper, incl. land application',
       'Biogasification of sewage slugde, incl. land application'],
      dtype='object', name='sector')

In [4]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Extraction of natural gas and services related to natural gas extraction, excluding surveying",
    "Manufacture of gas; distribution of gaseous fuels through mains",
    "Extraction, liquefaction, and regasification of other petroleum and gaseous materials",
    "Transport via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


In [5]:
total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
total_EU_imports_Z

region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   9,845.67
        Manufacture of gas; distribution of gaseous fuels through mains                                     6.92
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials               0.00
        Transport via pipelines                                                                           188.41
dtype: float64

In [10]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)
res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum() 

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res

region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials             0.00
        Transport via pipelines                                                                         100.00
dtype: float64


region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
AT                                                   0.01                                             \
BE                                                   0.34                                              
BG                                                   0.07                                              
CY                                                   0.00                                              
CZ                                                  13.19                                              
DE                                                   0.64                                              
DK                                                   0.27                                              
EE                                                   1.18                                              
ES                                                   0.20                                              
FI                                                   0.30                                              
FR                                                   0.40                                              
GR                                                   0.59                                              
HR                                                   0.01                                              
HU                                                  10.93                                              
IE                                                   0.33                                              
IT                                                  64.79                                              
LT                                                   2.56                                              
LU                                                   0.01                                              
LV                                                   2.42                                              
MT                                                   0.47                                              
NL                                                   0.11                                              
PL                                                   0.54                                              
PT                                                   0.02                                              
RO                                                   0.13                                              
SE                                                   0.22                                              
SI                                                   0.23                                              
SK                                                   0.05                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
AT                                                   1.06               \
BE                                                   5.04                
BG                                                   1.48                
CY                                                   0.79                
CZ                                                   6.46                
DE                                                  16.49                
DK                                                   0.42                
EE                                                   3.90                
ES                                                   2.71        

In [11]:
total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
total_EU_imports_Y

region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   3,175.29
        Manufacture of gas; distribution of gaseous fuels through mains                                     4.25
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials               0.00
        Transport via pipelines                                                                             2.45
dtype: float64

In [12]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)
res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum() 

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res

region  sector                                                                                       
RU      Extraction of natural gas and services related to natural gas extraction, excluding surveying   100.00
        Manufacture of gas; distribution of gaseous fuels through mains                                 100.00
        Extraction, liquefaction, and regasification of other petroleum and gaseous materials             0.00
        Transport via pipelines                                                                         100.00
dtype: float64


region                                                                                            RU   
sector Extraction of natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                                 
AT                                                   0.01                                             \
BE                                                   0.29                                              
BG                                                  17.70                                              
CY                                                   0.01                                              
CZ                                                   0.03                                              
DE                                                   1.16                                              
DK                                                   0.22                                              
EE                                                   0.11                                              
ES                                                   0.09                                              
FI                                                   0.07                                              
FR                                                   0.58                                              
GR                                                  22.75                                              
HR                                                   0.02                                              
HU                                                   0.84                                              
IE                                                   0.14                                              
IT                                                   0.54                                              
LT                                                   0.05                                              
LU                                                   0.02                                              
LV                                                   0.30                                              
MT                                                   0.47                                              
NL                                                   0.30                                              
PL                                                   0.26                                              
PT                                                   0.00                                              
RO                                                   9.61                                              
SE                                                   0.09                                              
SI                                                   2.60                                              
SK                                                  41.73                                              

region                                                                   
sector Manufacture of gas; distribution of gaseous fuels through mains   
region                                                                   
AT                                                   1.62               \
BE                                                   6.47                
BG                                                   2.32                
CY                                                   2.24                
CZ                                                   5.12                
DE                                                  22.75                
DK                                                   0.29                
EE                                                   1.98                
ES                                                   2.76        

In [13]:
# THE CASE OF GERMANY (INTER INDUSTRY)
# print(Z.loc["RU", "DE"].sum(axis=1).to_string())
Z.loc["RU", "DE"].sum(axis=1)

sector
Cultivation of paddy rice                              0.00
Cultivation of wheat                                   8.43
Cultivation of cereal grains nec                      13.37
Cultivation of vegetables, fruit, nuts                 6.50
Cultivation of oil seeds                              16.26
                                                       ... 
Activities of membership organisation n.e.c. (91)      0.44
Recreational, cultural and sporting activities (92)   18.67
Other service activities (93)                          0.01
Private households with employed persons (95)          0.00
Extra-territorial organizations and bodies             0.00
Length: 163, dtype: float64

In [14]:
# THE CASE OF GERMANY (FINAL DEMAND)
# print(Y.loc["RU", "DE"].sum(axis=1).to_string())
Y.loc["RU", "DE"].sum(axis=1)

sector
Cultivation of paddy rice                              0.00
Cultivation of wheat                                   3.18
Cultivation of cereal grains nec                       3.86
Cultivation of vegetables, fruit, nuts                14.56
Cultivation of oil seeds                               9.40
                                                       ... 
Activities of membership organisation n.e.c. (91)      0.15
Recreational, cultural and sporting activities (92)    2.03
Other service activities (93)                          0.04
Private households with employed persons (95)          0.00
Extra-territorial organizations and bodies             0.00
Length: 163, dtype: float64

#### PRODUCT x PRODUCT Table

In [2]:
# ---------------------------------------------------------
# SPECIFY PATH
dir_path = "../data/exiobase/_pxp/IOT_2019_pxp/"


# ---------------------------------------------------------
# IMPORT MATRICES

# Import A, Y, F and F_Y (from SATELLITE)
# ! 'A' is a large datasets, mind load time
Z = pd.read_csv(f"{dir_path}Z.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
A = pd.read_csv(f"{dir_path}A.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])
Y = pd.read_csv(f"{dir_path}Y.txt", delimiter="\t", header=[0, 1], index_col=[0, 1])

# Get X reference from Exiobase
x_exio = pd.read_csv(
    f"{dir_path}x.txt", delimiter="\t", header=0, index_col=[0, 1]
)  # ! header = 0, not [0,1]

# ! we use the satellite folder this time because we will look at uncharacterized stressors (CO2 emission vs GWP100)
F = pd.read_csv(
    f"{dir_path}satellite/F.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_y = pd.read_csv(
    f"{dir_path}satellite/F_Y.txt", delimiter="\t", header=[0, 1], index_col=[0]
)
F_unit = pd.read_csv(
    f"{dir_path}satellite/unit.txt", delimiter="\t", header=0, index_col=[0]
)

# ---------------------------------------------------------
# GET LABEL

# get multiindex
region_and_sector_labels = A.index

# get each level using  'get_level_values()'
region_labels = A.index.get_level_values(level=0).unique()
sector_labels = A.index.get_level_values(level=1).unique()

In [3]:
MASK = sector_labels.str.contains("gas", flags=re.IGNORECASE)
sector_labels[MASK]

Index(['Manure (biogas treatment)',
       'Natural gas and services related to natural gas extraction, excluding surveying',
       'Natural Gas Liquids', 'Gas Coke', 'Motor Gasoline',
       'Aviation Gasoline', 'Gasoline Type Jet Fuel', 'Gas/Diesel Oil',
       'Refinery Gas', 'Liquefied Petroleum Gases (LPG)', 'Biogasoline',
       'Electricity by gas', 'Coke oven gas', 'Blast Furnace Gas',
       'Oxygen Steel Furnace Gas', 'Gas Works Gas', 'Biogas',
       'Distribution services of gaseous fuels through mains',
       'Food waste for treatment: biogasification and land application',
       'Paper waste for treatment: biogasification and land application',
       'Sewage sludge for treatment: biogasification and land application'],
      dtype='object', name='sector')

In [4]:
EU27 = region_labels[:27]
RU = ["RU"]
biogas = ["Biogas"]
nat_gas = [
    "Natural gas and services related to natural gas extraction, excluding surveying",
    "Distribution services of gaseous fuels through mains",
    "Natural Gas Liquids",
    "Transportation services via pipelines",
]

EU27_nat_gas = pd.MultiIndex.from_product(iterables=[EU27, nat_gas])
RU_nat_gas = pd.MultiIndex.from_product(iterables=[RU, nat_gas])


In [5]:
total_EU_imports_Z = Z.loc[RU_nat_gas, EU27].sum(axis=1)
total_EU_imports_Z

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   10,431.67
        Distribution services of gaseous fuels through mains                                   0.00
        Natural Gas Liquids                                                                    0.00
        Transportation services via pipelines                                                169.99
dtype: float64

In [6]:
# How much does each EU country relies on DIRECT RU gas sectors (INTER INDUSTRY)
res = Z.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum() 

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Z, axis=0).T * 100
print(res.sum())
res

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                                 0.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
AT                                                   0.00                               \
BE                                                   0.00                                
BG                                                   0.04                                
CY                                                   0.00                                
CZ                                                  13.92                                
DE                                                   0.00                                
DK                                                   0.00                                
EE                                                   1.07                                
ES                                                   0.00                                
FI                                                   0.00                                
FR                                                   0.00                                
GR                                                   0.03                                
HR                                                   0.00                                
HU                                                  11.45                                
IE                                                   0.00                                
IT                                                  68.17                                
LT                                                   2.67                                
LU                                                   0.00                                
LV                                                   2.41                                
MT                                                   0.00                                
NL                                                   0.00                                
PL                                                   0.00                                
PT                                                   0.00                                
RO                                                   0.01                                
SE                                                   0.00                                
SI                                                   0.22                                
SK                                                   0.03                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
AT                                                    NaN    \
BE                                                    NaN     
BG                                                    NaN     
CY                                                    NaN     
CZ                                                    NaN     
DE                                                    NaN     
DK                                                    NaN     
EE                                                    NaN     
ES                                                    NaN     
FI                                                    NaN     
FR                                                    NaN     
GR                                                    NaN     
HR                                                    NaN     
HU                                                    NaN     
IE                                                    NaN     
IT                                                    NaN     
LT                                                    NaN     
LU                                     

In [7]:
total_EU_imports_Y = Y.loc[RU_nat_gas, EU27].sum(axis=1)
total_EU_imports_Y

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   3,374.79
        Distribution services of gaseous fuels through mains                                  0.00
        Natural Gas Liquids                                                                   0.00
        Transportation services via pipelines                                                 0.19
dtype: float64

In [8]:
# How much does each EU country relies on DIRECT RU gas sectors (FINAL DEMAND)
res = Y.loc[RU_nat_gas, EU27].groupby(level=0, axis=1).sum() 

# expressed in percentage of total EU imports
res = res.divide(total_EU_imports_Y, axis=0).T * 100
print(res.sum())
res

region  sector                                                                         
RU      Natural gas and services related to natural gas extraction, excluding surveying   100.00
        Distribution services of gaseous fuels through mains                                0.00
        Natural Gas Liquids                                                                 0.00
        Transportation services via pipelines                                             100.00
dtype: float64


region                                                                              RU   
sector Natural gas and services related to natural gas extraction, excluding surveying   
region                                                                                   
AT                                                   0.00                               \
BE                                                   0.00                                
BG                                                  18.61                                
CY                                                   0.00                                
CZ                                                   0.01                                
DE                                                   0.00                                
DK                                                   0.00                                
EE                                                   0.00                                
ES                                                   0.00                                
FI                                                   0.00                                
FR                                                   0.00                                
GR                                                  23.26                                
HR                                                   0.00                                
HU                                                   0.85                                
IE                                                   0.00                                
IT                                                   0.41                                
LT                                                   0.00                                
LU                                                   0.00                                
LV                                                   0.08                                
MT                                                   0.00                                
NL                                                   0.00                                
PL                                                   0.00                                
PT                                                   0.00                                
RO                                                  10.09                                
SE                                                   0.00                                
SI                                                   2.71                                
SK                                                  43.98                                

region                                                        
sector Distribution services of gaseous fuels through mains   
region                                                        
AT                                                    NaN    \
BE                                                    NaN     
BG                                                    NaN     
CY                                                    NaN     
CZ                                                    NaN     
DE                                                    NaN     
DK                                                    NaN     
EE                                                    NaN     
ES                                                    NaN     
FI                                                    NaN     
FR                                                    NaN     
GR                                                    NaN     
HR                                                    NaN     
HU                                                    NaN     
IE                                                    NaN     
IT                                                    NaN     
LT                                                    NaN     
LU                                     

In [9]:
# # THE CASE OF GERMANY (INTER INDUSTRY)
# print(Z.loc["RU", "DE"].sum(axis=1).to_string())
Z.loc["RU", "DE"].sum(axis=1)

sector
Paddy rice                                           0.00
Wheat                                                1.65
Cereal grains nec                                   10.64
Vegetables, fruit, nuts                              2.75
Oil seeds                                           14.03
                                                     ... 
Membership organisation services n.e.c. (91)         0.00
Recreational, cultural and sporting services (92)    7.89
Other services (93)                                  0.01
Private households with employed persons (95)        0.00
Extra-territorial organizations and bodies           0.00
Length: 200, dtype: float64

In [10]:
# THE CASE OF GERMANY (FINAL DEMAND)
# print(Y.loc["RU", "DE"].sum(axis=1).to_string())
Y.loc["RU", "DE"].sum(axis=1)

sector
Paddy rice                                           0.00
Wheat                                                2.79
Cereal grains nec                                    3.51
Vegetables, fruit, nuts                             13.29
Oil seeds                                            9.22
                                                     ... 
Membership organisation services n.e.c. (91)         0.00
Recreational, cultural and sporting services (92)    0.01
Other services (93)                                  0.04
Private households with employed persons (95)        0.00
Extra-territorial organizations and bodies           0.00
Length: 200, dtype: float64

# Comparison with EUROSTATS

![Russian imports](../img/Eurostat_russian_imports.png)

Import of Russian Natural Gas (in TJ). Source: [Eurostats](https://ec.europa.eu/eurostat/databrowser/view/NRG_TI_GAS__custom_6448417/default/table?lang=en)
